Test de stationnarité & fit with good modele selection

# Cours : Séries Temporelles et Modèles ARMA

## 1. Conditions de Stationnarité et Dépendance Faible

### Stationnarité
Une série temporelle est **stationnaire** si elle respecte les trois conditions suivantes :
1. **Moyenne constante** : 
   $E[X_t] = \mu \neq \mu(t)$  
   La moyenne est une constante finie indépendante du temps.
   
2. **Variance constante** : 
   $\text{Var}[X_t] = \sigma^2 \neq \sigma^2(t)$  
   La variance est une constante finie indépendante du temps.

3. **Covariance indépendante du temps** : 
   $\text{Cov}(X_t, X_{t+h}) = \gamma(h)$  
   La covariance dépend uniquement du lag $h$, et non du temps $t$.

### Dépendance faible
Pour qu'une série soit faiblement dépendante :  
$\text{Corr}(X_t, X_{t+h}) \to 0 \text{ lorsque } h \to \infty$

---

## 2. Processus de Moyenne Mobile (MA)

### Définition
Un processus **MA(q)** s'écrit :  
$
X_t = \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots + \theta_q \epsilon_{t-q}
$
où $\epsilon_t \sim \text{iid}(0, \sigma^2)$.



### Stationnarité
Tout processus MA est toujours stationnaire.

---

## 3. Processus Auto-Régressif (AR)

### Définition
Un processus **AR(q)** s'écrit :  
$
X_t = \phi_1 X_{t-1} + \phi_2 X_{t-2} + \dots + \phi_q X_{t-q} + \epsilon_t
$
où $\epsilon_t \sim \text{iid}(0, \sigma^2)$.

### Stationnarité
Un processus AR(q) est stationnaire si les racines du polynôme caractéristique :  
$
z^q - \phi_1 z^{q-1} - \phi_2 z^{q-2} - \dots - \phi_q = 0
$
sont **dans le cercle unité** $|z| < 1$.

---

## 4. Auto-Corrélation et Corrélation Partielle

### Auto-Corrélation
La fonction d'auto-corrélation totale mesure la corrélation entre $X_t$ et $X_{t-h}$ :
$
\text{Corr}(X_t, X_{t-h}) = \frac{\text{Cov}(X_t, X_{t-h})}{\sqrt{\text{Var}[X_t] \text{Var}[X_{t-h}]}}
$

### Corrélation Partielle
La corrélation partielle à un lag $h$ est la corrélation entre $X_t$ et $X_{t-h}$, en contrôlant les effets des lags intermédiaires $(1, 2, \dots, h-1)$. Elle peut être calculée en résolvant les **équations de Yule-Walker**.

### Différences
- **Corrélation totale** : Mesure l'effet global, y compris les dépendances indirectes.
- **Corrélation partielle** : Mesure uniquement l'effet direct entre deux lags.

---

## 5. Valeurs-p (P-values)

### Définition
Une **valeur-p** représente la probabilité d'obtenir un résultat aussi extrême que celui observé, sous l'hypothèse nulle ($H_0$). Elle est liée à la fonction de répartition cumulative (CDF).

### Exemple
- Une pièce donnée produit 100% de faces sur 10 lancers : $p = 0.001$.  
- Une autre pièce produit 51% de faces sur 1 million de lancers : $p \approx 10^{-89}$.

La valeur-p seule ne quantifie pas le biais ; il faut considérer les **intervalles de confiance**.

---

## 6. Processus ARMA

### Définition
Un processus **ARMA(p, q)** combine un modèle AR(p) et MA(q) :
$
X_t = \phi_1 X_{t-1} + \dots + \phi_p X_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q}
$

### Propriétés
- **Stationnarité** : Dépend des racines du polynôme AR.
- **Invertibilité** : Dépend des racines du polynôme MA.

### Exemple
Les ventes $X_t$ peuvent être modélisées par un processus ARMA(1,1) :
$
X_t = 0.5 X_{t-1} + \epsilon_t - 0.3 \epsilon_{t-1}
$
- AR(1) pour les effets de fidélité.
- MA(1) pour les effets résiduels immédiats.

---

## Conclusion

Ce cours couvre les bases des modèles AR, MA, et ARMA, ainsi que des concepts clés comme la stationnarité, les corrélations et les valeurs-p. Ces outils sont essentiels pour l'analyse des séries temporelles et la modélisation prédictive.


In [3]:
import statsmodels.api as sm

# ARIMA is more general model that include ARMA, it has 3 parameters
from statsmodels.tsa.arima.model import ARIMA

# for Lyung Box test of autocorrelation
from statsmodels.stats.diagnostic import acorr_ljungbox

# For the augmented Dickey-Fuller test
from statsmodels.tsa.stattools import adfuller

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Step by Step: 
1. Load the time-series
2. Plot the time-series
3. Check the stationarity
4. If it is stationary, continue; if not, take the first difference and go to the step 3.
5. Plot ACF and PACF 
6. Identify possible models (p,q)
7. Fit all models that you identified
8. Sort by AIC and measure ∆AIC. Exclude some models.
9. Look at the confidence intervals on the estimated parameters (We would like to have all significants)
10. Look at the ACF of the residues for the candidate models
11. Test with Ljungbox if a serial correlation remains

3. Check the stationarity

on plot l'autocorrelation (ACF function) et on regarde.
- si l'autocorrelation diminue doucement $=>$ marker of non-stationnarity

In [ ]:
s1= # time serie

sm.graphics.tsa.plot_acf(s1, lags=200)
plt.ylabel(r'$lags$')
plt.xlabel('ACF [$X_t$]')

* On s'en asure avec le Augmented Dickey-Fuller test to check for stationarity in the time series

In [ ]:
adfuller(s1)

le Augmented Dickey-Fuller test ( ADF tet) renvoie un tuple comprenant (Test Statistic ; P-value ; Number of lags Used ; Number of Observations used ; Critical values ; AIC maximum information Criterion)

As example, if the ADF test for a time-serie give the values : 
1. Test statistic = -2.6645643672746036 
2. p-value = 0.08037345141097707 it represent the probability of obtaining a test statistic at least as exreme as the one observed, assuming the null hypothesis at the 1% significante level 
3. Number of lags used = 4 
4. Number of observations used = 9994 
5. Critical Value = {'1%': -3.4310044907347454,'5%': -2.8618292459530794,'10%': -2.566923960483154}
6. Maximum information criterion (AIC) = -63471.61889044968


The statistic test = -2.6645643672746036 is greater than the critical value at the level of 1% = '1%': -3.4310044907347454 . So, we fail to reject the null hypothesis at the 1% significant level, indicating that the serie is non-stationnary 

* 4. Differencing
to adress the non-stationnarity in the contexte of ARIMA is used to differencing to turned into stationnary series. Differencing is a common method to make a time serie stationnary.

We apply first-order differencing (d=1) and we check again the autocorrelation and we do the ADF test again

In [ ]:
s1=np.diff(s1)

On réitere jusqu'à ce que la p-value soit plus petite que la critical value.

Le nombre de differenciation faites permet  d'identifier l'ARIMA Model $(p, \text{nb de diff}, q)$ 

* 6. Identify possible models (p,q)
the steps are <br> 1| Determine AR(p) (by analizing the ACF) and MA(q) (by analizing the PACF). Keep in mind that While the ACF suggests a higher-order MA component, it is often beneficial to start with simpler models. simpler model are easier to interpret and less prome to overfitting.<br> 2| Fit ARMA model to the differenced model using the different p, q identified earlly


In [ ]:
models = [(4,0),(1,1),(2,1),(1,2),(2,2)]

M={}
for p,q in models:
    print(p,q)
    ft=ARIMA(s1,order=(p,1,q)).fit()
    M[p,q]=ft

# Comparing Model Using AIC
AIC = pd.DataFrame([((p,q),ft.aic) for (p,q), ft in M.items()], columns=['model','AIC'])
print(AIC)

AIC = AIC.assign(dAIC=(AIC.AIC - AIC.AIC.min()))
AIC.sort_values('AIC')

Models with dAIC<2 are considered indistinguishable from the best model <br> for dAIC < 7 it's ok but we prefere dAIC <2

on analyse le meilleur model $(p, \text{nb de diff}, q)$ 

We can simplificate the model by removing the non-significant constant term

In [ ]:
Best_modèle=(p,d,q)
model_ar4_no_const=ARIMA(s1, order=(4,0,0), trend='n')
results_ar4_no_const=model_ar4_no_const.fit()
results_ar4_no_const.summary()

 3| Model diagnostics : evaluated the model using various diagnostic

 We have to check the residuals to ensure that the model is appropriate

r=results_ar4_no_const.resid

sm.graphics.tsa.plot_acf(r, lags=20)
plt.ylabel(r'$lags$')
plt.xlabel('ACF [residus]')

Ljung-Box Test, used to confirm the adequacy of the model, we perform the ljung-box test on the residuals. this test check if there are any significant autocorrelations remaining in the residuals

In [ ]:
acorr_ljungbox(r,lags=range(p+q+10), model_df=p+q,return_df=True)